In [1]:
"""
Vesuvius competition metric.

Expects standard Kaggle paths and Linux in order to manage dependencies.
"""

import glob
import importlib
import os
import subprocess
import sys
import numpy as np
import pandas as pd
from PIL import Image, ImageSequence

class ParticipantVisibleError(Exception):
    pass

class HostVisibleError(Exception):
    pass

In [2]:
def load_volume(path):
    im = Image.open(path)
    slices = []
    for i, page in enumerate(ImageSequence.Iterator(im)):
        slice_array = np.array(page)
        slices.append(slice_array)
    volume = np.stack(slices, axis=0)
    return volume

In [ ]:
def install_dependencies():
    """On Kaggle, the topometrics library must be installed during the run. This function handles the entire process."""
    try:
        import topometrics.leaderboard

        return None
    # The broad exception is necessary as the initial import can fail for multiple reasons.
    except:
        pass

    resources_dir = '/projects/nian/synthrad2025/Vesuvius/src/vesuvius-metric-resources'
    install_dir = '/projects/nian/synthrad2025/Vesuvius/src/topological-metrics-kaggle'

    try:
        subprocess.run(
            f'cd {resources_dir} && uv pip install --no-index --find-links=wheels -r topological-metrics-kaggle/requirements.txt',
            shell=True,
            check=True,
        )
        subprocess.run(f'cd /projects/nian/synthrad2025/Vesuvius/src && cp -r {resources_dir}/topological-metrics-kaggle .', shell=True, check=True)
        subprocess.run(
            f'cd {install_dir} && chmod +x scripts/setup_submodules.sh scripts/build_betti.sh && make build-betti',
            shell=True,
            check=True,
        )
        subprocess.run(
            f'cd {install_dir} && uv pip install -e . --no-deps --no-index --no-build-isolation -v',
            shell=True,
            check=True,
        )
        # Add the new library to Python's path and invalidate caches to ensure it's found.
        sys.path.append('/projects/nian/synthrad2025/Vesuvius/src/topological-metrics-kaggle/src')
        importlib.invalidate_caches()

    except Exception as err:
        raise HostVisibleError(f'Failed to install topometrics library: {err}')


In [9]:
def generate_standard_submission(submission_dir: str) -> None:
    # Dependencies installed here as generate_standard_submission is the first metric function that gets called by the orchestrator.
    submission_tifs = glob.glob(f'{submission_dir}/**/*.tif', recursive=True)
    if len(submission_tifs) == 0:
        submission_tifs = glob.glob('/kaggle/tmp/**/*.tif', recursive=True)
    if len(submission_tifs) == 0:
        raise ParticipantVisibleError('No submission files found')
    df = pd.DataFrame({'tif_paths': submission_tifs})
    df['id'] = df['tif_paths'].apply(lambda x: x.split('/')[-1].split('.')[0])
    os.chdir('/kaggle/working')
    df[['id', 'tif_paths']].to_csv('submission.csv', index=False)

In [10]:
def score_single_tif(
    gt_path,
    pred_path,
    surface_tolerance,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
    ):
    gt: np.ndarray = load_volume(gt_path)
    pr: np.ndarray = load_volume(pred_path)

    # install_dependencies() # TODO uncomment
    # The import is here to ensure dependencies are loaded first.
    try:
        # Use a standard import now that the path is reliably set.
        import topometrics.leaderboard
    except Exception as err:
        raise HostVisibleError(f'Failed to import topometrics after installation: {err}')

    score_report = topometrics.leaderboard.compute_leaderboard_score(
        predictions=pr,
        labels=gt,
        dims=(0, 1, 2),
        spacing=(1.0, 1.0, 1.0),  # (z, y, x)
        surface_tolerance=surface_tolerance,  # in spacing units
        voi_connectivity=voi_connectivity,
        voi_transform=voi_transform,
        voi_alpha=voi_alpha,
        combine_weights=(topo_weight, surface_dice_weight, voi_weight),  # (Topo, SurfaceDice, VOI)
        fg_threshold=None,  # None => legacy "!= 0"; else uses "x > threshold"
        ignore_label=2,  # voxels with this GT label are ignored
        ignore_mask=None,  # or pass an explicit boolean mask
    )
    return np.clip(score_report.score, a_min=0.0, a_max=1.0)


In [11]:
def score(
    solution: pd.DataFrame,
    submission: pd.DataFrame,
    row_id_column_name: str,
    surface_tolerance: float = 2.0,
    voi_connectivity: int = 26,
    voi_transform: str = 'one_over_one_plus',
    voi_alpha: float = 0.3,
    topo_weight: float = 0.3,
    surface_dice_weight: float = 0.35,
    voi_weight: float = 0.35,
    ) -> float:
    """Returns the mean per-volume Topological Score, Surface Dice, and VOI Scores."""
    if not solution['tif_paths'].apply(os.path.exists).all():
        raise HostVisibleError('Invalid solution file paths')

    solution['pred_paths'] = submission['tif_paths']
    solution['image_score'] = solution.apply(
        lambda row: score_single_tif(
            row['tif_paths'],
            row['pred_paths'],
            surface_tolerance,
            voi_connectivity=voi_connectivity,
            voi_transform=voi_transform,
            voi_alpha=voi_alpha,
            topo_weight=topo_weight,
            surface_dice_weight=surface_dice_weight,
            voi_weight=voi_weight,
        ),
        axis=1,
    )
    return float(np.mean(solution['image_score']))


In [12]:
install_dependencies()

Using Python 3.11.14 environment at: /homes/andre.ferreira/.conda/envs/topo-metrics-3d
Audited 11 packages in 53ms


./scripts/setup_submodules.sh
./scripts/build_betti.sh
Using:
  Python_EXECUTABLE=/homes/andre.ferreira/.conda/envs/topo-metrics-3d/bin/python
  Python_INCLUDE_DIR=/homes/andre.ferreira/.conda/envs/topo-metrics-3d/include/python3.11
  Python_LIBRARY=/homes/andre.ferreira/.conda/envs/topo-metrics-3d/lib/libpython3.11.so


CMake Deprecation Warning at CMakeLists.txt:3 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.




-- The C compiler identification is GNU 14.3.0
-- The CXX compiler identification is GNU 14.3.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /homes/andre.ferreira/.conda/envs/topo-metrics-3d/bin/x86_64-conda-linux-gnu-cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /homes/andre.ferreira/.conda/envs/topo-metrics-3d/bin/x86_64-conda-linux-gnu-c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Python: /homes/andre.ferreira/.conda/envs/topo-metrics-3d/bin/python (found suitable version "3.11.14", minimum required is "3.7") found components: Interpreter Development.Module Development.Embed
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- Found pybind11: /homes/andre.ferreira/.conda/envs/topo-metrics-3d/include (found ve

/homes/andre.ferreira/.conda/envs/topo-metrics-3d/bin/../lib/gcc/x86_64-conda-linux-gnu/14.3.0/../../../../x86_64-conda-linux-gnu/bin/ld: warning: size of symbol `' changed from 3 in CMakeFiles/BettiMatching.dir/src/main.cpp.o to 2067 in CMakeFiles/BettiMatching.dir/src/main.cpp.o
/homes/andre.ferreira/.conda/envs/topo-metrics-3d/bin/../lib/gcc/x86_64-conda-linux-gnu/14.3.0/../../../../x86_64-conda-linux-gnu/bin/ld: warning: NOTE: size discrepancies can cause real problems.  Investigation is advised.
/homes/andre.ferreira/.conda/envs/topo-metrics-3d/bin/../lib/gcc/x86_64-conda-linux-gnu/14.3.0/../../../../x86_64-conda-linux-gnu/bin/ld: CMakeFiles/BettiMatching.dir/src/main.cpp.o: in function `no symbol':
:(.text.startup.main+0x0): multiple definition of `no symbol'; CMakeFiles/BettiMatching.dir/src/main.cpp.o::(.text._Z20print_usage_and_exiti+0x0): first defined here
/homes/andre.ferreira/.conda/envs/topo-metrics-3d/bin/../lib/gcc/x86_64-conda-linux-gnu/14.3.0/../../../../x86_64-conda-

gmake[3]: Leaving directory '/projects/nian/synthrad2025/Vesuvius/src/topological-metrics-kaggle/external/Betti-Matching-3D/build'
[100%] Linking CXX shared module betti_matching.cpython-311-x86_64-linux-gnu.so
gmake[3]: Leaving directory '/projects/nian/synthrad2025/Vesuvius/src/topological-metrics-kaggle/external/Betti-Matching-3D/build'
gmake[2]: Leaving directory '/projects/nian/synthrad2025/Vesuvius/src/topological-metrics-kaggle/external/Betti-Matching-3D/build'
gmake[1]: Leaving directory '/projects/nian/synthrad2025/Vesuvius/src/topological-metrics-kaggle/external/Betti-Matching-3D/build'


/homes/andre.ferreira/.conda/envs/topo-metrics-3d/bin/../lib/gcc/x86_64-conda-linux-gnu/14.3.0/../../../../x86_64-conda-linux-gnu/bin/ld: CMakeFiles/betti_matching.dir/src/data_structures.cpp.o: plugin needed to handle lto object
/homes/andre.ferreira/.conda/envs/topo-metrics-3d/bin/../lib/gcc/x86_64-conda-linux-gnu/14.3.0/../../../../x86_64-conda-linux-gnu/bin/ld: CMakeFiles/betti_matching.dir/src/data_structures.cpp.o: .symtab local symbol at index 2 (>= sh_info of 2)
/homes/andre.ferreira/.conda/envs/topo-metrics-3d/bin/../lib/gcc/x86_64-conda-linux-gnu/14.3.0/../../../../x86_64-conda-linux-gnu/bin/ld: CMakeFiles/betti_matching.dir/src/data_structures.cpp.o: error adding symbols: bad value
collect2: error: ld returned 1 exit status
gmake[3]: *** [CMakeFiles/betti_matching.dir/build.make:453: betti_matching.cpython-311-x86_64-linux-gnu.so] Error 1
gmake[2]: *** [CMakeFiles/Makefile2:90: CMakeFiles/betti_matching.dir/all] Error 2
gmake[1]: *** [Makefile:91: all] Error 2
make: *** [Mak

HostVisibleError: Failed to install topometrics library: Command 'cd /projects/nian/synthrad2025/Vesuvius/src/topological-metrics-kaggle && chmod +x scripts/setup_submodules.sh scripts/build_betti.sh && make build-betti' returned non-zero exit status 2.

In [7]:
score_single_tif(
    gt_path="/projects/nian/synthrad2025/Vesuvius/DataSet/example/train_labels/1407735.tif",
    pred_path="/projects/nian/synthrad2025/Vesuvius/DataSet/example/train_labels/1407735.tif",
    surface_tolerance=2,
    voi_connectivity=26,
    voi_transform='one_over_one_plus',
    voi_alpha=0.3,
    topo_weight=0.3,
    surface_dice_weight=0.35,
    voi_weight=0.35,
    )

HostVisibleError: Failed to import topometrics after installation: Found /projects/nian/synthrad2025/Vesuvius/src/metrics/topological-metrics-kaggle/external/Betti-Matching-3D/build but could not import 'betti_matching'. Make sure you've built the submodule (make build-betti). Original error: No module named 'betti_matching'